In [1]:
import pandas as pd
import numpy as np

import miditoolkit

import re
import glob
import os

from music21 import converter, midi, key, instrument, meter, interval, note, pitch, stream, tempo, chord
import os
import pandas as pd
import glob

from tqdm import tqdm

from utils.utils import open_midi, get_file_name, extract_tempo, extract_key, to_snake_case, chord_type_checker

from difflib import get_close_matches

In [2]:
p_df = pd.read_csv(r'/Users/hai/Desktop/hch/projects/midi/csv/program_change.csv')
com_df = pd.read_csv(r'/Users/hai/Desktop/hch/projects/midi/csv/commu_meta.csv', index_col=0)
df = pd.read_csv(r'/Users/hai/Desktop/hch/projects/midi/csv/allgenre_cp.csv')

chord_df = pd.read_csv(r'/Users/hai/Desktop/hch/projects/midi/csv/chord_list.csv')

In [3]:
scale = ["maj", "maj7","7","dim","dim7","+","m","m7","sus4","7sus4","m7b5","sus2","add2","madd2","mM7", '5', '6' , 'm5' , 'm6']
chord_df['CHORD_TYPE'] = chord_df['CHORD_TYPE'].apply(lambda x: '+' if x == 'aug' else x)
chord_df['CHORD_TYPE'] = chord_df['CHORD_TYPE'].apply(lambda x: 'mM7' if x == 'm(maj7)' else x)
chord_df['CHORD_TYPE'] = chord_df['CHORD_TYPE'].apply(lambda x: 'add2' if x == 'add9' else x)
chord_df = chord_df[chord_df['CHORD_TYPE'].isin(scale)]
chord_df.reset_index(drop=True, inplace=True)

chord_df['Notes_list'] = chord_df['NOTES'].apply(lambda x: [y for y in x.split(',')])
chord_df['Notes_list_string'] = chord_df['Notes_list'].apply(lambda x: str(x))

In [4]:
commu_folder = './commu_midi'
com_m = glob.glob(f'{commu_folder}/*.mid')
com_m.sort()

In [5]:
our_m = list()
for idx, v in enumerate(df['song_title'].values):

    genre = df['genre'].values[idx]
    data_folder = './data/' + genre + '/cropped'

    our_m.append(os.path.join(data_folder, v + '/' + df['file_name'].values[idx]))

In [6]:
def extract_chord_progression(midi_path, chord_df, cl):

    score = converter.parse(midi_path)

    chord_progression = list()
    cp = list()

    for i in range(8):
        
        note_list = list()

        for obj in score.measures(i+1, i+1).recurse():
            if isinstance(obj, note.Note):
                if obj.duration.quarterLength > 0.3:
                    note_list.append(obj.name)  
            elif isinstance(obj, chord.Chord):
                for p in obj.pitches:   
                    note_list.append(p.name) 

        current_chord = list()

        for value in note_list:
            if value not in current_chord:
                current_chord.append(value)

        if len(current_chord) == 0:
            cp.append('Bar_None')

        if len(current_chord) == 1:
            cp.append(str(current_chord[0]))

        if len(current_chord) > 1:

            notes = str()
            
            for i in current_chord:
                notes += i
            
            if len(notes) > 5:
                notes = notes[:5]
            
            close_matches = get_close_matches(notes, cl , 1, 0.6)
            
            if not close_matches:
                close_matches = get_close_matches(notes, cl , 1, 0.5)

            tnl = list()
            for i in range(len(str(close_matches[0]))):
                if close_matches[0][i] == '#':
                    tnl.append(close_matches[0][i-1] + '#')
                    tnl.remove(close_matches[0][i-1])
                elif close_matches[0][i] == '-':
                    tnl.append(close_matches[0][i-1] + 'b')
                    tnl.remove(close_matches[0][i-1])
                else:
                    tnl.append(close_matches[0][i])

            basic = chord_df[chord_df['Notes_list_string'] == str (tnl)]['CHORD_ROOT'].values[0]
            chord_type = chord_df[chord_df['Notes_list_string'] == str(tnl)]['CHORD_TYPE'].values[0]
            if chord_type == '5':
                chord_type = ''
            if chord_type == 'm5':
                chord_type = 'm'
            cp.append(str(basic + chord_type))

    for c in cp:
        for i in range(8):
            chord_progression.append(c)
    
    return chord_progression

In [7]:
df['chord_progressions'] = None

cl = chord_df['NOTES'].to_list()
for idx, v in enumerate(cl):
    cl[idx] = v.replace(',', '')

for idx, v in enumerate(cl):
    cl[idx] = v.replace('b', '-')

for idx, path in tqdm(enumerate(our_m)):
    
    chord_progression = extract_chord_progression(path, chord_df, cl)
    df.at[idx, 'chord_progressions'] = [chord_progression]
    

60401it [25:13, 39.92it/s]


In [10]:
# df.to_csv(r'/Users/hai/Desktop/hch/projects/midi/csv/allgenre_cp.csv', index=False)